In [1]:
OUTPUT_FILE = "output_yeast_structure.csv"
INPUT_FILE = '/home/imhof_team/Public/mauricio/workflow/yeast_type3/yeast_type3_safe2.fasta'

import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

#import experiments
import seqdhbm.SeqDHBM as SeqDHBM
import seqdhbm.fasta as fasta
import seqdhbm.pdbfiles as pdbfiles
import sys, os
import shutil
import time

def workflow(jobnum= "J0", fastafile=None, pdbs=[], pdbid="", rawseq="", mode="structure"):
    """ Runs the workflow 
    
    Params:
    - fasta: One fasta file
    - pdbfiles: list of pdb filenames, they should be in the root dir
    - pdbid: string with PDB codes (comma separated)
    - knowstruc: boolean: we use services to obtain the structure if False"""
    # TODO: use jobnum to organize submissions
    # Deal with the fasta file with multiple sequences
    seq_dict = {}
    
    seq_list = []
    if fastafile:
        """seq_dict = fasta.fasta_to_seq(fastafile)
        fasta.organize_sequences(seq_dict)"""
        seq_list = fasta.fasta_to_seq2(fastafile)
        fasta.organize_sequences2(seq_dict)
        #logging.debug("fasta:")
        #logging.debug(seq_dict.keys())

    # Deal with the pdb ids (by downloading them)
    pdbid_dict = {} # stores the pdbfile path to each pdb id
    if pdbid:
        raise Exception("not prepared to work with they new variables")
        l = pdbfiles.text_to_list(pdbid)
        pdbid_dict = pdbfiles.get_pdb_files(l)
        #logging.debug("pdbid")
        #logging.debug(pdbid_dict)
    
    if rawseq:
        seq_dict["Your input sequence"] = ("Your input sequence header", rawseq)
        folder = "%s/MI/"%jobnum
        file = folder+"MI1.fasta"
        name = "Your input sequence"
        """assert False, "create the folder first" """
        os.makedirs(folder, exist_ok=True)
        with open(file, "w") as f:
            f.write(">%s\n"%name)
            lines=[rawseq[x:x+60] for x in range(0, len(rawseq), 60)]
            f.write('\n'.join(lines))
        seq_list += [{"seq":rawseq, 
                     "name":name, 
                     "folder": folder,
                     "file": file,
                     "submited_as": "Manual input"}]
    
    # Arrange the pdb files (user-submitted or downloaded)
    ## Put the structure file in folders - for docking and MD
    ## get the primary sequence for motif search
    for file in pdbs:
        raise Exception("not prepared to work with they new variables")
        try:
            folder = ".".join(file.split(".")[:-1]) # remove extension
        except:
            folder = file
        try:
            os.makedirs(folder, exist_ok=True)
            shutil.move(file, folder+"/"+file)
            pdbid_dict[folder] = folder+"/"+file
        except:
            logging.error("Could not move file"+ file + "\nSkiping it!") # TODO: List of warnings and errors
            # TODO: organize the files into folders
    
    # TODO: convert the pdbs into fasta
    for file in pdbid_dict.values():
        seq = pdbfiles.structure_to_fasta(file)
        if seq:
            seq_dict[file] = (">header", seq)
    # Run the motif check
    #logging.debug("*"*30)
    #logging.debug(seq_dict)
    
    #spacerinfo = {}
    
    hbm_result = {}
    cnt_progress = 0
    seq_dict = {}
    for item in seq_list:
        cnt_progress+=1
        try:
            analysis = SeqDHBM.SpotCoordinationSite({">"+item["name"]: item["seq"]}, mode)
            analysedSeq = analysis[">" + item["name"]]

            # Output? Save in our user output?
            item["result"] = analysedSeq["result"]
            item["warnings"] = analysedSeq["warnings"]

        except Exception as e:
            logging.error("****")
            logging.error("Sequence name:%s \n %s"%(item["name"], item["file"]))
            logging.error(e)
            logging.error(type(e))
            logging.error(e.args)
            logging.error("****")
        finally:
            logging.debug("Got the results for %s"%item["name"])
            logging.debug("%d out of %d complete"%(cnt_progress, len(seq_dict)))

    return seq_list


if (__name__ == "__main__"):

    fastafile = ''
    mode = ""
    if sys.argv[0].endswith("ipykernel_launcher.py"):
        fastafile = ""# INPUT_FILE
        rawseq = "ABCDZEFGH,"
        mode = "structure"
        scriptname = "workflow" # TODO: Will this be the final name?
    else:
        scriptname = os.path.basename(__file__)
    pdbids = ""# "1S0L,1L2H,1I1B,1HIB,1I8H"
    pdbs=[]
    
    for params in sys.argv:
        if params.startswith("-pdbid="):
            pdbids = params[7:]
        elif params.startswith("-fasta="):
            fastafile = params[7:]
        elif params.startswith("-pdbfiles="):
            # param pdbid must be formatted as list
            pdbs = eval(params[10:])
        elif params.startswith("-mode="):
            # param pdbid must be formatted as list
            mode = params[6:].lower()
        elif params.startswith("-help"):
            print("Usage Help")
            print("call by using 'python %s -mode=<mode> -fasta=<fastafile> -pdbid=<pdb_id> -pdbfiles=<pdb_files>'"%(scriptname))
            print("<mode> is mandatory and accepts:")
            print("-mode=structure: if structure is known (default)")
            print("-mode=wesa: to use our services to PREDICT surface accessibility")
            print("<fastafile> is the full path of a fasta file with one or more sequences")
            print("<pdb_id> must contain the PDB ids, separated by commas.")
            print("<pdb_files> PDB files with .ent or .pdb format")
    
    assert mode in ["structure", "wesa"], ("Mode not detected!\n"
    "Use 'python %s' for help on using the application\n")
            
    workflow(jobnum=0, fastafile=fastafile, pdbs=pdbs, rawseq=rawseq,pdbid=pdbids, mode=mode)



INFO - --------------------------
INFO - SEQUENCE VALIDITY CHECK:
INFO - --------------------------
INFO - NOTE : 0 sequences out of 1 are valid
INFO - --------------------
INFO - INVALID SEQUENCE(S)
INFO - --------------------
INFO - NOTE : The following sequence(s) contains invalid characters other than the 20 standard amino acids 

INFO - 1.Your input sequence
WARNING - NOTE : ALL SEQUENCES IN YOUR FILE CONTAIN ONE OR MORE INVALID CHARACTERS !!
WARNING - NOTE : PLEASE CHECK YOUR INPUT SEQUENCE(S) AND TRY AGAIN !
WARNING - NOTE : PROGRAM WILL EXIT NOW !
DEBUG - Got the results for Your input sequence
DEBUG - 1 out of 0 complete
